In [72]:
%env PROJ_LIB=/opt/conda/share/proj

env: PROJ_LIB=/opt/conda/share/proj


In [12]:
#%env

In [13]:
#os.environ['PROJ_LIB'] = '/opt/anaconda3/envs/test/share/proj'

In [74]:
from mpl_toolkits.basemap import Basemap
import pandas as pd
import matplotlib.pyplot as plt
import math
import matplotlib.animation as animation
# Missing Migrant file
loc = 'MissingMigrants-Global-2019-06-30T06-47-14.csv'
# Required fields, drop row if they don't exist
drop=['Location Coordinates']
# Filter regions
regions = ['US-Mexico Border','Central America','South America','North America']
%matplotlib notebook

In [15]:
# Some settings for the plot
width = 2500000
height = 1600000
centerlat = 30
centerlon = -105

In [16]:
# Read in the data, dropping NaN from the Location Coordinates
mig = pd.read_csv(loc,parse_dates=['Reported Date']).dropna(subset=drop)

In [17]:
# Split out the latitude and longitude
mig['Latitude'] = mig['Location Coordinates'].str[0:14]
mig['Longitude'] = mig['Location Coordinates'].str[17:]
# Filter to the desired regions
mig = mig[mig['Region of Incident'].isin(regions)]
# Convert Total Dead and Missing to int
mig['Total Dead and Missing'] = mig['Total Dead and Missing'].astype('int64')
mig['Number of Children'] = mig['Number of Children'].fillna(0).astype('int64')

In [18]:
#mig['Region of Incident'].value_counts()

In [19]:
mig['Reported Date'].min()

Timestamp('2014-01-06 00:00:00')

In [20]:
mig.dtypes

Web ID                                          int64
Region of Incident                             object
Reported Date                          datetime64[ns]
Reported Year                                   int64
Reported Month                                 object
Number Dead                                   float64
Minimum Estimated Number of Missing           float64
Total Dead and Missing                          int64
Number of Survivors                           float64
Number of Females                             float64
Number of Males                               float64
Number of Children                              int64
Cause of Death                                 object
Location Description                           object
Information Source                             object
Location Coordinates                           object
Migration Route                                object
URL                                            object
UNSD Geographical Grouping  

In [21]:
### TODO Explode out dataframe
# Each row should be one death
# Each needs lat/long, Child/Adult, and Date
# Then add a jitter to the lat/long

In [22]:
# Get a count of children and non-children
long = mig[['Total Dead and Missing','Number of Children','Reported Date','Latitude','Longitude']]
long['Non-Child Dead and Missing'] = long['Total Dead and Missing'] - long['Number of Children']

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [23]:
# Use list comprehension to get a single row for every adult
adult = pd.DataFrame([long.loc[idx] 
                       for idx in long.index 
                       for _ in range(long.loc[idx]['Non-Child Dead and Missing'])]).reset_index(drop=True)
# Same for every child
child = pd.DataFrame([long.loc[idx] 
                       for idx in long.index 
                       for _ in range(long.loc[idx]['Number of Children'])]).reset_index(drop=True)

In [24]:
long['Number of Children'].sum()

110

In [25]:
#adult

In [26]:
# Filter out the edges of map
#top = centerlat + ((height/2)/111111)
#bottom = centerlat - ((height/2)/111111)
#left = centerlon + ((width/2)/111111*math.cos(centerlat))
#right = centerlon - ((width/2)/111111*math.cos(centerlat))

In [27]:
#mig[mig['Total Dead and Missing'] > '5'][['Web ID','Total Dead and Missing','Number of Females','Number of Males', 'Number of Children']]

In [28]:
#mig[mig['Web ID'] == 39383]

In [29]:
#mig.groupby('Reported Date')['Number Dead'].sum().sort_values()

In [30]:
mig.columns

Index(['Web ID', 'Region of Incident', 'Reported Date', 'Reported Year',
       'Reported Month', 'Number Dead', 'Minimum Estimated Number of Missing',
       'Total Dead and Missing', 'Number of Survivors', 'Number of Females',
       'Number of Males', 'Number of Children', 'Cause of Death',
       'Location Description', 'Information Source', 'Location Coordinates',
       'Migration Route', 'URL', 'UNSD Geographical Grouping',
       'Source Quality', 'Latitude', 'Longitude'],
      dtype='object')

In [31]:
mig['Location Coordinates'][1][17:]

'-99.696202800000'

In [32]:
mig['Number of Children'].sum()

110

In [33]:
# Calculate middle lat from center to top edge
conf1 = centerlat + ((height/4)/111111)
# Calculate middle lat from center to bottom edge
conf2 = centerlat - ((height/4)/111111)

In [36]:
# TODO
adult.head()
lons = adult[adult['Reported Date'] < '2019-06-26']['Longitude']
#lats = 

In [48]:
child[child['Reported Date'] > '2019-01-01']['Reported Date']

0    2019-06-24
1    2019-06-23
2    2019-06-23
3    2019-06-23
4    2019-06-14
5    2019-06-12
6    2019-05-21
7    2019-05-14
8    2019-05-14
9    2019-05-13
10   2019-05-05
11   2019-05-05
12   2019-05-01
13   2019-05-01
14   2019-05-01
15   2019-04-30
16   2019-04-07
17   2019-03-24
18   2019-03-07
19   2019-03-07
20   2019-03-07
21   2019-03-07
22   2019-03-07
23   2019-03-05
24   2019-02-24
25   2019-01-28
26   2019-01-28
27   2019-01-28
28   2019-01-28
29   2019-01-28
30   2019-01-28
31   2019-01-28
32   2019-01-28
33   2019-01-28
34   2019-01-28
35   2019-01-24
Name: Reported Date, dtype: datetime64[ns]

In [59]:
lons

0      -97.497454400000
1      -98.328768700000
2      -98.328768700000
3      -98.328768700000
4      -94.135662914059
5     -106.337425829110
6     -111.324478000000
7      -98.118324500000
8     -106.485021700000
9      -92.821729096217
10     -92.259368268791
11     -92.259368268791
12    -100.939984373880
13    -100.939984373880
14    -100.939984373880
15     -97.386347400000
16     -99.662936105483
17     -99.478838604955
18     -92.916367624797
19     -92.916367624797
20     -92.916367624797
21     -92.916367624797
22     -92.916367624797
23     -48.480863500000
24     -48.480863500000
25      77.330689179762
26      77.330689179762
27      77.330689179762
28      77.330689179762
29      77.330689179762
30      77.330689179762
31      77.330689179762
32      77.330689179762
33      77.330689179762
34      77.330689179762
35    -106.426767800000
36    -105.933164337420
37    -100.511538546900
38    -117.014062400000
39    -117.014062400000
40    -106.499407000000
41    -105.47799

In [76]:
# Static map for development

# setup Lambert Conformal basemap.
m = Basemap(resolution='l',
            # Lamber Conformal projection
            projection='lcc',
            width=width,
            height=height,
            lat_1=conf1,
            lat_2=conf2,
            lat_0=centerlat,
            lon_0=centerlon)
# I think I'll leave the country lines out, to emphasize the arbitrary nature of boundaries.
#m.drawcountries(linewidth=0.5)
# Blue marble is the prettiest map, in contrast to the ugliness of the data
m.bluemarble()
# Pull out the latitude and longitude for the proper period
lons = child[child['Reported Date'] > '2018-01-01']['Longitude'].values
lats = child[child['Reported Date'] > '2018-01-01']['Latitude'].values
lons2 = adult[adult['Reported Date'] > '2018-01-01']['Longitude'].values
lats2 = adult[adult['Reported Date'] > '2018-01-01']['Latitude'].values
x,y=m(lons,lats)
m.plot(x,y,'bo',markersize=6)
x,y=m(lons2,lats2)
m.plot(x,y,'ro',markersize=6)
plt.show()